# 模型微调流程
需要unsloth环境

## 导入模型
修改模型路径

In [ ]:
from unsloth import FastLanguageModel
import torch
import os
from datasets import load_dataset
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
max_seq_length = 8192
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "C:/Users/23756/models/Qwen3-4B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\23756\miniconda3\envs\unsloth\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\23756\miniconda3\envs\unsloth\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:341: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.6.5: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.26s/it]


## 导入数据集
此处应为数据集处理脚本

In [3]:
reasoning_dataset = load_dataset(data_files="./datasets/ReasonMed.json", split = "train", path="json")

In [4]:
reasoning_dataset[0]

{'instruction': 'Please answer the following multiple-choice question:\nA factory worker presents with excessive salivation, blue lines on gums, tremors, disturbed personality, insomnia, and loss of appetite. The most likely poisoning is -?\nA. Mercury\nB. Lead\nC. Arsenic\nD. Phosphorus',
 'input': '',
 'output': "<think>\nThe patient is a factory worker presenting with excessive salivation, blue lines on the gums, tremors, disturbed personality, insomnia, and loss of appetite. These symptoms collectively suggest a form of heavy metal poisoning. To determine the most likely poisoning, it is essential to meticulously analyze each symptom in the context of the provided options: Mercury, Lead, Arsenic, and Phosphorus.\n\nExcessive salivation, also known as ptyalism, is a significant symptom to consider. This symptom is notably associated with mercury poisoning, particularly with elemental mercury exposure, which can disrupt autonomic functions leading to increased salivation. Tremors, es

In [ ]:
import json

# 假定你已经有了 reasoning_dataset，例如通过 json.load 读取
# reasoning_dataset = json.load(open('reasoning_dataset.json', 'r', encoding='utf-8'))

dataset = {"conversations": []}

for sample in reasoning_dataset:
    user_msg = {
        "role": "user",
        "content": sample["instruction"] + (("\n" + sample["input"]) if sample.get("input") else "")
    }
    assistant_msg = {
        "role": "assistant",
        "content": sample["output"]
    }
    dataset["conversations"].append([user_msg, assistant_msg])


<class 'dict'> 369983
[{'role': 'user', 'content': 'Please answer the following multiple-choice question:\nA factory worker presents with excessive salivation, blue lines on gums, tremors, disturbed personality, insomnia, and loss of appetite. The most likely poisoning is -?\nA. Mercury\nB. Lead\nC. Arsenic\nD. Phosphorus'}, {'role': 'assistant', 'content': "<think>\nThe patient is a factory worker presenting with excessive salivation, blue lines on the gums, tremors, disturbed personality, insomnia, and loss of appetite. These symptoms collectively suggest a form of heavy metal poisoning. To determine the most likely poisoning, it is essential to meticulously analyze each symptom in the context of the provided options: Mercury, Lead, Arsenic, and Phosphorus.\n\nExcessive salivation, also known as ptyalism, is a significant symptom to consider. This symptom is notably associated with mercury poisoning, particularly with elemental mercury exposure, which can disrupt autonomic functions 

In [9]:
# 验证
print(type(dataset), len(dataset["conversations"]))
print(dataset["conversations"][1])

<class 'dict'> 369983
[{'role': 'user', 'content': "Please answer the following multiple-choice question:\nRett's syndrome occurs due to deficiency of ?\nA. Niacin\nB. Biotin\nC. Carotene\nD. Vit D"}, {'role': 'assistant', 'content': "<think>\nRett's syndrome is a severe neurodevelopmental disorder that predominantly affects females and is characterized by a regression in cognitive and motor skills after a period of apparently normal development. It typically manifests in early childhood, with symptoms becoming noticeable between the ages of 6 to 18 months. The condition involves a range of neurological symptoms, including loss of purposeful hand movements, slowed growth, seizures, and difficulties with communication and social interaction. Importantly, Rett's syndrome is a genetic disorder caused by mutations in the MECP2 gene, which is located on the X chromosome. This gene plays a critical role in brain development and function by regulating the expression of other genes essential f

In [10]:
final_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)
import pandas as pd
data = pd.concat([
    pd.Series(final_reasoning_conversations),
])
data.name = "text"
from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

In [12]:
combined_dataset

Dataset({
    features: ['text'],
    num_rows: 369983
})

## 模型训练

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_num_proc = 1,
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = None, # Use this for WandB etc
    ),
)

Unsloth 2025.6.5 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.
Unsloth: Tokenizing ["text"]: 100%|██████████| 369983/369983 [04:36<00:00, 1340.08 examples/s]


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 369,983 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288/4,000,000,000 (1.65% trained)


Step,Training Loss
1,1.377900
2,1.397500
3,1.230300
4,1.097300
5,1.288400
6,1.001000
7,1.161000
8,1.041900
9,1.088300
10,0.946600


## 模型保存
注意修改路径，保存为符合HF规范的文件夹

In [15]:
# 保存 LoRA adapter 权重
model.save_pretrained("Qwen3-4b-v0.2")

# 保存 tokenizer（可以跳过，如果没有改动过）
tokenizer.save_pretrained("Qwen3-4b-v0.2")

('Qwen3-4b-v0.2\\tokenizer_config.json',
 'Qwen3-4b-v0.2\\special_tokens_map.json',
 'Qwen3-4b-v0.2\\chat_template.jinja',
 'Qwen3-4b-v0.2\\vocab.json',
 'Qwen3-4b-v0.2\\merges.txt',
 'Qwen3-4b-v0.2\\added_tokens.json',
 'Qwen3-4b-v0.2\\tokenizer.json')

In [16]:
from peft import LoraConfig, PeftModel, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer


base_model_path = "C:/Users/23756/models/Qwen3-4B"
lora_model_path = "./Qwen3-4b-v0.2"  # 你训练后保存的路径
merged_model_path = "./merged-Qwen3-4b-v0.2"  # 合并后保存路径
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,       # 根据实际显存可选 float16/float32
    low_cpu_mem_usage=True
).to(device)

tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 2. 从 LoRA adapter 加载增量权重
print("Loading LoRA adapter...")
peft_model = PeftModel.from_pretrained(
    base_model,
    lora_model_path,
    torch_dtype=torch.float16
)

# 3. 合并 LoRA 到基础模型
print("Merging LoRA weights into the base model...")
merged_model = peft_model.merge_and_unload()

# 4. 保存合并后的模型和分词器
print(f"Saving merged model to {merged_model_path} ...")
merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

print("Done! 你现在可以直接使用 merged-Qwen3-4b-v0.2 目录下的模型进行推理。")

Loading checkpoint shards: 100%|██████████| 3/3 [01:03<00:00, 21.28s/it]


Loading LoRA adapter...
Merging LoRA weights into the base model...
Saving merged model to ./merged-Qwen3-4b-v0.2 ...
Done! 你现在可以直接使用 merged-Qwen3-4b-v0.2 目录下的模型进行推理。


## 模型量化
将HF格式的模型转换为GGUF格式的模型，便于vllm或ollama部署

执行命令将合并后HF模型转换为bin文件
cd C:/Users/23756/demo/llama.cpp
python convert_hf_to_gguf.py C:/Users/23756/demo/merged-Qwen3-4b-v0.2 --outtype f16 --outfile C:/Users/23756/demo/convert.bin

再将bin文件量化为gguf文件(参数自选)
C:/Users/23756/demo/llama.cpp/build/bin/Release/llama-quantize C:/Users/23756/demo/convert.bin q6_k

## 对HF格式的模型进行简单测试

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "merged-Qwen3-4b-v0.2"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input


In [2]:
prompt = "该患者是一名工厂工人，表现出流涎过多（即唾液分泌过多）、牙龈蓝线、震颤、性格改变、失眠和食欲减退等症状。这些症状综合起来提示可能为某种重金属中毒。为了确定最可能的中毒类型，我们需要仔细分析这些症状，并结合选项：汞、铅、砷和磷。"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
好的，我现在要分析这位工厂工人的症状，看看他可能是什么重金属中毒。首先，他有流涎过多，牙龈蓝线，震颤，性格改变，失眠和食欲减退。这些症状让我联想到不同的重金属中毒情况。

流涎过多可能和汞中毒有关，因为汞中毒确实会导致唾液分泌增加，但牙龈蓝线这个症状我之前没怎么听说过。震颤和性格改变的话，汞中毒确实会有这些表现，不过我需要再想想其他可能性。

铅中毒的话，通常会有腹痛、便秘、贫血和神经症状，比如头痛、易怒、注意力不集中，但流涎和牙龈蓝线好像不太对。不过铅中毒也可能导致震颤，所以不能完全排除。

砷中毒的话，症状包括胃肠道不适、皮肤损害、神经系统问题，还有可能引起震颤和性格改变，但流涎和牙龈蓝线好像不太符合。不过砷中毒确实会有失眠和食欲减退，所以也不能完全排除。

磷中毒的话，主要症状是胃肠道症状，比如恶心、呕吐，还有可能引起神经症状，但流涎和牙龈蓝线似乎不太符合。不过磷中毒也可能导致震颤和性格改变，所以也不能完全排除。

现在我需要再仔细想想这些症状之间的关联。流涎过多和牙龈蓝线可能更倾向于汞中毒，因为汞中毒确实会导致唾液分泌增加和牙龈蓝线，而震颤和性格改变可能与汞中毒有关。不过我之前对牙龈蓝线这个症状不太确定，可能需要再查证一下。

另外，铅中毒的话，虽然可能有震颤和性格改变，但流涎和牙龈蓝线不太符合。砷中毒的话，虽然有失眠和食欲减退，但其他症状也不太符合。磷中毒的话，主要症状是胃肠道不适，所以可能不太符合。

综合来看，汞中毒的可能性最大，因为流涎过多和牙龈蓝线更符合汞中毒的症状，而震颤和性格改变也有可能。不过我需要再确认一下牙龈蓝线是否确实是汞中毒的一个特征症状。

另外，考虑到这位工人在工厂工作，可能接触到汞或其他重金属，所以需要进一步考虑他的工作环境。如果他在工厂里接触到汞，那么汞中毒的可能性就更高了。

不过，我还需要考虑其他可能性，比如是否还有其他重金属中毒的症状符合这些表现。比如，铅中毒可能有震颤和性格改变，但流涎和牙龈蓝线不太符合。砷中毒可能有失眠和食欲减退，但其他症状也不太符合。

所以，综合所有因素，最可能的中毒类型是汞中毒，因为流涎过多和牙龈蓝线更符合汞中毒的症状，而震颤和性格改变也有可能。不过我需要再确认一下这些症状是否确实与汞中毒有关。
</think>
content: 这位工厂工人表